In [5]:
# =========================================================
# 🎓 AutoAttendance — Combined API + Test Mode
# =========================================================
# Toggle this between "api" or "test"
MODE = "api"   # "test" for local testing without Cloudflare/Sheets

# --- Step 1: Install Dependencies ---
!pip install -q deepface gspread google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb > /dev/null

# --- Step 2: Imports ---
from flask import Flask, request, jsonify
from threading import Thread
import subprocess, time, re, os, cv2, numpy as np, socket
from deepface import DeepFace
from datetime import date

# --- Step 3: Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- Step 4: Common Config ---
BASE_DIR = "/content/drive/MyDrive/AttendanceProject"
ENCODE_DIR = f"{BASE_DIR}/encodings"
CLASS_PHOTOS_DIR = f"{BASE_DIR}/class_photos"
MODEL_NAME = "Facenet"
SIMILARITY_THRESHOLD = 0.6

# --- Step 5: Optional Google Sheet Setup (Only in API Mode) ---
if MODE == "api":
    import gspread
    from google.colab import auth
    from google.auth import default
    SHEET_NAME = "AutoAttendanceDB"

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    sheet = gc.open(SHEET_NAME).sheet1
    print(f"✅ Connected to Google Sheet: {SHEET_NAME}")
else:
    print("🧪 Running in TEST mode — no Google Sheets update.")

# --- Step 6: Load Encodings ---
def load_embeddings():
    if not os.path.exists(ENCODE_DIR):
        raise FileNotFoundError(f"❌ Folder not found: {ENCODE_DIR}")
    encodings = {}
    for file in os.listdir(ENCODE_DIR):
        if file.endswith(".npy"):
            name = file.replace(".npy", "")
            encodings[name] = np.load(os.path.join(ENCODE_DIR, file))
    print(f"✅ Loaded {len(encodings)} encodings.")
    return encodings

encodings = load_embeddings()

# --- Step 7: Face Recognition Function ---
def recognize_students(photo_path, threshold=SIMILARITY_THRESHOLD):
    detections = DeepFace.extract_faces(img_path=photo_path, detector_backend='retinaface', enforce_detection=False)
    print(f"🧩 {len(detections)} faces detected in photo.")
    present = []
    for i, det in enumerate(detections):
        face_img = (det['face'] * 255).astype(np.uint8)
        tmp_path = f"/content/temp_face_{i}.jpg"
        cv2.imwrite(tmp_path, face_img)
        rep = DeepFace.represent(img_path=tmp_path, model_name=MODEL_NAME, enforce_detection=False)[0]['embedding']
        rep = np.array(rep)
        best_name, best_score = None, -1
        for name, ref_emb in encodings.items():
            sim = float(np.dot(rep, ref_emb) / (np.linalg.norm(rep) * np.linalg.norm(ref_emb)))
            if sim > best_score:
                best_name, best_score = name, sim
        if best_score >= threshold:
            present.append(best_name)
        print(f"🧍 Face {i+1}: {best_name} ({best_score:.3f})")
    present = list(set(present))
    print("✅ Recognized:", present)
    return present

# --- Step 8: Mark Attendance (API Mode Only) ---
def mark_attendance(present_students):
    if MODE == "test":
        print("🧪 [TEST] Attendance not written to Google Sheets.")
        return []
    headers = sheet.row_values(1)
    records = sheet.get_all_records()
    today = date.today().strftime("%Y-%m-%d")
    if today not in headers:
        sheet.update_cell(1, len(headers)+1, today)
        headers.append(today)
    date_col = headers.index(today) + 1
    absentees = []
    for row_idx, rec in enumerate(records, start=2):
        student_name = str(rec.get("Name", "")).strip()
        if student_name in present_students:
            sheet.update_cell(row_idx, date_col, "Present")
        else:
            sheet.update_cell(row_idx, date_col, "Absent")
            absentees.append(student_name)
    return absentees

# =========================================================
# --- Step 9: Flask API Setup ---
# =========================================================
from flask import Flask, jsonify, request
app = Flask(__name__)

@app.route('/')
def home():
    return f"✅ AutoAttendance API ({MODE.upper()} MODE) is live!"

@app.route('/process', methods=['POST'])
def process():
    try:
        data = request.get_json()
        photo_name = data.get("photo_name")
        photo_path = os.path.join(CLASS_PHOTOS_DIR, photo_name)
        if not os.path.exists(photo_path):
            return jsonify({"error": f"Photo not found: {photo_name}"}), 404

        print(f"\n📸 Processing photo: {photo_path}")
        present_students = recognize_students(photo_path)
        absentees = mark_attendance(present_students)

        return jsonify({
            "status": "success",
            "present": present_students,
            "absent": absentees
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# =========================================================
# --- Step 10: Port Safe Start + Cloudflare (API Only) ---
# =========================================================
import socket, re, time, subprocess
def get_free_port(start_port=5001):
    port = start_port
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(('localhost', port)) != 0:
                return port
            port += 1

PORT = get_free_port(5001)

def run_flask():
    app.run(host='0.0.0.0', port=PORT)

if MODE == "api":
    !pkill -f cloudflared 2>/dev/null || true
    !kill -9 $(lsof -t -i:5001 -i:5002) 2>/dev/null || true

    from threading import Thread
    Thread(target=run_flask, daemon=True).start()

    def get_public_url():
        !pkill cloudflared 2>/dev/null
        !cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate > cf.log 2>&1 &
        time.sleep(8)
        with open("cf.log") as f:
            log = f.read()
        url = re.search("https://[-0-9a-z]*\\.trycloudflare\\.com", log)
        if url:
            return url.group(0)
        return None

    public_url = get_public_url()
    if public_url:
        print(f"\n🌍 Public URL: {public_url}")
        print(f"🔹 Test endpoint: {public_url}/process")
    else:
        print("❌ Could not retrieve Cloudflare URL. Try re-running this cell.")
else:
    print(f"🧪 Flask running locally on port {PORT}")
    app.run(host='0.0.0.0', port=PORT)


# --- Save the public Cloudflare URL to Google Sheet ---
def update_api_url_in_sheet(public_url):
    try:
        config_sheet = gc.open(SHEET_NAME).worksheet("Config")
        # ✅ Correct usage: update the cell value directly
        config_sheet.update_acell('B2', f"{public_url}/process")
        print(f"✅ Updated API URL in Config sheet: {public_url}/process")
    except Exception as e:
        print(f"⚠️ Could not update API URL in sheet: {e}")

        # --- Call the function to actually update the sheet ---
if MODE == "api" and 'public_url' in locals() and public_url:
    update_api_url_in_sheet(public_url)
else:
    print("⚠️ No public URL available to update in Google Sheet.")



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
25-11-01 04:43:31 - Directory /root/.deepface has been created
25-11-01 04:43:31 - Directory /root/.deepface/weights has been created
Mounted at /content/drive
✅ Connected to Google Sheet: AutoAttendanceDB
✅ Loaded 2 encodings.
^C
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit



🌍 Public URL: https://blond-retain-declared-armed.trycloudflare.com
🔹 Test endpoint: https://blond-retain-declared-armed.trycloudflare.com/process
✅ Updated API URL in Config sheet: https://blond-retain-declared-armed.trycloudflare.com/process
